# Embedded kernels 

This notebook and the C# project in this folder demonstrates how you can use .NET Interactive to embed a kernel within an app, connect to it from another kernel, and use the notebook to change the app's runtime state.



## Connect to the WPF app

First, let's start the WPF app and connect to it.

In [4]:
Start-Process -NoNewWindow dotnet run

Once the cell above has finished running, you should see the app's window open. Next, we'll connect to it using a named pipe. The code that sets this up within the WPF app can be seen in [`App.xaml.cs`](https://github.com/dotnet/interactive/blob/main/samples/connect-wpf/App.xaml.cs).

In [5]:
#r "C:\Projects\interactive\artifacts\bin\Microsoft.DotNet.Interactive.NamedPipeConnector\Debug\netstandard2.0\Microsoft.DotNet.Interactive.NamedPipeConnector.dll"
Microsoft.DotNet.Interactive.NamedPipeConnector.ConnectNamedPipeCommand.AddToRootKernel();

In [ ]:
#!connect named-pipe --kernel-name wpf --pipe-name InteractiveWpf

The topology of connected kernels now looks like this:

In [3]:
flowchart LR
    subgraph WPF app
    embedded["Embedded C# kernel"]
    end
    subgraph notebook
    CompositeKernel-->n1["Local C# kernel"]
    CompositeKernel-->n2
    n2["#!wpf kernel added using #!connect"]--named pipe-->embedded
    end

## Change the styling of the app

The notebook outputs here are displayed using custom formatters defined within the WPF app itself. Take a look at the file [`WpfFormatterMixins.cs`](https://github.com/dotnet/interactive/blob/main/samples/connect-wpf/WpfFormatterMixins.cs).

You'll also notice that you can get completions for the `App` object which is exposed to the notebook's kernel by the embedded kernel. 

In [ ]:
#!dispatcher
using System.Windows.Media;

App.MainWindow.Background = new SolidColorBrush(Colors.Fuchsia);
App.MainWindow.Background

In [ ]:
#!dispatcher
using System.Windows.Media;
using System.Windows.Controls;
using System.Windows;

var content = (Grid)App.MainWindow.Content;
content.Background = new SolidColorBrush(Colors.RoyalBlue);
content.UpdateLayout();
content

## Change view models at runtime



Create and apply a new view model to the main window.

In [9]:
using System.ComponentModel;
using System.Collections.ObjectModel;

public class TestViewModel : INotifyPropertyChanged
{
    public event PropertyChangedEventHandler PropertyChanged;

    private string _text = "Initial Value from notebook view model";
    
    public string Text
    {
        get => _text;
        set
        {
            if (_text != value)
            {
                _text = value;
                PropertyChanged.Invoke(this, new PropertyChangedEventArgs(nameof(Text)));
            }
        }
    }
}

var vm = new TestViewModel();

#!wpf
#!dispatcher
App.MainWindow.DataContext = vm;

Update the value on the data bound property.

In [10]:
vm.Text = "Value changed!"

 ## Dispatcher stuff

 Demonstate enabling and disabling running code on the dispatcher. 

In [13]:
#!dispatcher --enabled 
//This should work
App.MainWindow.Title = "Title change executed on dispatcher thread";


In [14]:

#!dispatcher --enabled false
//This is expected to fail
App.MainWindow.Title = "Not so much";